# Data Preprocessing

__In this notebook, we will go through the steps followed in order to prepare (preprocess) the data required for our application. <br>
The data used in this user study is provided by the well-known online recipe website [allrecipes](https://www.allrecipes.com/). <br>
The data is meant to use for research purposes. <br>
In a nutshell, the steps are :__
  1. __Read the csv file of each category and add categpry column to DF__
  2. __Extract the recipes that has image, as images will be used in our study__
  3. __Compute Fsa and Nutri score for each recipe as main decision factor for healthiness of each recipe__
  4. __Split recipies into healthy and unhealthy ones based on Fsa threshold (8)__
  5. __Prepare rating matrices for each category, to be usd in recommendation generation phase__
  6. __save the data__
  7. __Optional: Push the data into Database__<br>
For more information about the dataset: __ayoub.majjodi@uib.no__

In [95]:
import pandas as pd
import numpy as np

In [96]:
# read data files
def read_data(data):
    sep = '\t'
    file = data + '.csv'
    if data == 'images':
        sep = ';'
    row_data = pd.read_csv(file, sep = sep)
    return row_data
# size of each dataset
def size(data, name):
    return   print('number of recipes in {0}: {1}'.format( name, len(data)))

In [97]:
# read each category
bbq = read_data('bbq')
fruit_veg = read_data('fruits_vegetables')
meat_poultry = read_data('Meat_poultry')
pasta_noodles = read_data('pasta_and_noodles')
salad = read_data('salad')
seafood = read_data('Seafood')
soups_chili = read_data('soups_stews_and_chili')
images = read_data('images')

In [98]:
size(bbq, 'bbq'), size(fruit_veg, 'fruit_veg')
size(meat_poultry, 'meat_poultry'), size(pasta_noodles, 'pasta_noodles')
size(salad, 'salad'), size(seafood, 'seafood')
size(soups_chili, 'soups_chili'), size(images, 'images')

number of recipes in bbq: 1682
number of recipes in fruit_veg: 19574
number of recipes in meat_poultry: 12672
number of recipes in pasta_noodles: 2692
number of recipes in salad: 3031
number of recipes in seafood: 3237
number of recipes in soups_chili: 3605
number of recipes in images: 162985


(None, None)

## add category to each data

In [99]:
bbq['category'] = 'Barbecue'
fruit_veg['category'] = 'Fruits and Vegetables'
meat_poultry['category'] = 'Meat and Poultry'
salad['category'] = 'Salad'
pasta_noodles['category'] = 'Pasta and Noodles'
seafood['category'] = 'Seafood'
soups_chili['category'] = 'Soups and Chili'

# extract recipes that has images

In [100]:
def recipe_with_image(data, images):
    images = images.drop_duplicates(subset=['recipe_id'])
    data_with_image = pd.merge(data, images, left_on=  'URL',
                   right_on= 'recipe_id', how='inner')
    if data_with_image.isnull().values.any():
           data_with_image= data_with_image.dropna()
    data_with_image =  data_with_image.drop(['cook_id','recipe_id','date_added'], axis=1)
    data_with_image['id'] = range(0, len(data_with_image))
    first_column = data_with_image.pop('id')
    data_with_image.insert(0,'id',first_column)
    return data_with_image

In [101]:
final_bbq = recipe_with_image(bbq, images)
final_fruit =  recipe_with_image(fruit_veg, images)
final_meat_poultry =  recipe_with_image(meat_poultry, images)
final_salad = recipe_with_image(salad, images)
final_pasta_noodles = recipe_with_image(pasta_noodles, images)
final_seafood = recipe_with_image(seafood, images)
final_soups_chili = recipe_with_image(soups_chili, images)

In [102]:
size(final_bbq, 'bbq'), size(final_fruit, 'fruit_veg')
size(final_meat_poultry, 'meat_poultry'), size(final_pasta_noodles, 'pasta_noodles')
size(final_salad, 'salad'), size(final_seafood, 'seafood')
size(final_soups_chili, 'soups_chili')

number of recipes in bbq: 1200
number of recipes in fruit_veg: 12916
number of recipes in meat_poultry: 8375
number of recipes in pasta_noodles: 1810
number of recipes in salad: 2062
number of recipes in seafood: 1931
number of recipes in soups_chili: 2199


In [103]:
final_bbq.to_csv('./Results/bbq.csv', index=False)
final_fruit.to_csv('./Results/fruit.csv', index=False)
final_meat_poultry.to_csv('./Results/meat_poultry.csv', index=False)
final_pasta_noodles.to_csv('./Results/pasta_noodles.csv',index=False)
final_salad.to_csv('./Results/salad.csv',index=False)
final_seafood.to_csv('./Results/seafood.csv',index=False)
final_soups_chili.to_csv('./Results/soup_chili.csv',index=False)

# Concat recipes

In [104]:
recipes = [final_bbq,final_fruit,final_meat_poultry,final_pasta_noodles,final_salad
          ,final_seafood,final_soups_chili]
final_recipes = pd.concat(recipes)
final_recipes.to_csv('./Results/final_recipes.csv', index=False)

In [105]:
final_recipes['category'].value_counts()

Fruits and Vegetables    12916
Meat and Poultry          8375
Soups and Chili           2199
Salad                     2062
Seafood                   1931
Pasta and Noodles         1810
Barbecue                  1200
Name: category, dtype: int64

# Preapre nutrients for 100g and Nuri/Fsa score

In [106]:
final_recipes = pd.read_csv('./Results/final_recipes.csv')
final_recipes.columns

Index(['id', 'URL', 'Name', 'Fiber (g)', 'Sodium (g)', 'Carbohydrates (g)',
       'Fat (g)', 'Protein (g)', 'Sugar (g)', 'Saturated Fat (g)', 'Size (g)',
       'Servings', 'Calories (kCal)', 'Average Rating', 'Average Sentiment',
       'Number of Ratings', 'Number of Bookmarks', 'Year of Publishing',
       'category', 'image_link'],
      dtype='object')

In [107]:
final_recipes['fiber_100g'] = (final_recipes['Servings'] * final_recipes['Fiber (g)'] * 100) / final_recipes['Size (g)']
final_recipes['fat_100g'] = (final_recipes['Servings'] * final_recipes['Fat (g)'] * 100) / final_recipes['Size (g)']
final_recipes['sugar_100g'] = (final_recipes['Servings'] * final_recipes['Sugar (g)'] * 100) / final_recipes['Size (g)']
final_recipes['saturated_100g'] = (final_recipes['Servings'] * final_recipes['Saturated Fat (g)'] * 100) / final_recipes['Size (g)']
final_recipes['sodium_100mg'] = ((final_recipes['Servings'] * final_recipes['Sodium (g)'] * 100) / final_recipes['Size (g)'])*1000
final_recipes['carbohydrates_100g'] = (final_recipes['Servings'] * final_recipes['Carbohydrates (g)'] * 100) / final_recipes['Size (g)']
final_recipes['protein_100g'] = (final_recipes['Servings'] * final_recipes['Protein (g)'] * 100) / final_recipes['Size (g)']
final_recipes['kj_100g'] = ((final_recipes['Servings'] * final_recipes['Calories (kCal)'] * 100) / final_recipes['Size (g)'])*4.1868
final_recipes['Salt (g)'] = final_recipes['Sodium (g)'] * 2.54
final_recipes['salt_100g'] = (final_recipes['Servings'] * final_recipes['Salt (g)'] * 100) / final_recipes['Size (g)']

In [108]:
# rename columns to match ones in database
final_recipes = final_recipes.rename(columns={'Fiber (g)':'fiber_g','Sodium (g)':'sodium_g','Carbohydrates (g)':'carbohydrates_g',
                                                           'Fat (g)':'fat_g','Protein (g)':'protein_g','Sugar (g)':'sugar_g',
                                                           'Saturated Fat (g)':'saturate_g','Size (g)':'size_g',
                                                           'Calories (kCal)':'calories_kCal', 'Salt (g)':'salt_g', 'Number of Ratings':'NumberRatings'})


# Fsa Score based on :
https://www.food.gov.uk/sites/default/files/media/document/fop-guidance_0.pdf

In [109]:
for i in final_recipes.index:
    fat_count = satfat_count = sugar_count = salt_count = 2
    if final_recipes.at[i,'fat_100g'] <= 3:
        fat_count = 1
    if final_recipes.at[i, 'fat_g'] > 21 or final_recipes.at[i, 'fat_100g'] > 17.5: # here the mistake :) > 17.5
        fat_count = 3
    if final_recipes.at[i,'saturated_100g'] <= 1.5:
        satfat_count = 1
    if final_recipes.at[i, 'saturate_g'] > 6 or final_recipes.at[i,'saturated_100g'] > 5:
        satfat_count = 3
    if final_recipes.at[i, 'sugar_100g'] <= 5:
        sugar_count = 1
    if final_recipes.at[i, 'sugar_100g'] > 22.5 or final_recipes.at[i, 'sugar_g'] > 27:
        sugar_count = 3
    if final_recipes.at[i, 'salt_100g'] <= 0.3:
        salt_count = 1
    if final_recipes.at[i, 'salt_100g'] > 1.5 or final_recipes.at[i, 'salt_g']> 1.8:
        salt_count = 3 
    final_recipes.at[i,'Fsa_new'] = fat_count + satfat_count + sugar_count + salt_count
    final_recipes.at[i, 'fat_count'] = fat_count
    final_recipes.at[i, 'satfat_count'] = satfat_count
    final_recipes.at[i, 'sugar_count'] = sugar_count
    final_recipes.at[i, 'salt_count'] = salt_count

In [110]:
final_recipes['Fsa_new'].value_counts().sort_index()

4.0     1422
5.0     2600
6.0     4780
7.0     4271
8.0     4808
9.0     4991
10.0    6519
11.0     895
12.0     207
Name: Fsa_new, dtype: int64

In [111]:
# Nutri score string function
def NutriScore_string(score):
    str_score = ''
    if score in range(-15, 0):
        str_score = 'A'
    if score in range(0,3):
        str_score = 'B'
    if score in range(3,11):
        str_score = 'C'
    if score in range(11,19):
        str_score = 'D'
    if score in range(19,41):
        str_score = 'E'
    return str_score    

## Compute nutri score based on 
https://www.santepubliquefrance.fr/content/download/150263/file/2021_07_21_QR_scientifique_et_technique_V41_EN.pdf <br>
Paper: __Development of a new front-of-pack nutrition label in France: the five-colour Nutri-Score__

In [112]:
# compute nutri- score
for i in final_recipes.index.to_list():
#     print(i)
    # assign a point to enrgyA
    energy = final_recipes.at[i,'kj_100g']
    if energy > 3350:
        kj_point = 10
    else:
        energy_level = energy % 335 # if the rest of is > 335 move to next point
        if energy_level <= 335:
            kj_point = energy // 335
        else:
            kj_point = (energy // 335) + 1
    # assign a point to Sugar
    sugar = final_recipes.at[i, 'sugar_100g']
    if sugar > 45:
        sg_point = 10
    else:
        sg_level = sugar % 4.5
        if sg_level <= 4.5:
            sg_point = sugar // 4.5
        else:
            sg_point = (sugar // 4.5) + 1
    # assign a point to saturated_100g
    sat_fat = final_recipes.at[i, 'saturated_100g']
    if sat_fat > 10:
        sf_point = 10
    else:
        sf_level = sat_fat % 1
        if sf_level <= 1:
            sf_point = sat_fat // 1
        else:
            sf_point = (sat_fat // 1) + 1
    # assign a point to sodium_100mg
    sodium = final_recipes.at[i, 'sodium_100mg']
    if sodium > 900:
        sdm_point = 10
    else:
        sdm_level = sodium % 90
        if sdm_level <= 90:
            sdm_point = sodium // 90
        else:
            sdm_point = (sodium // 90) + 1

    N_point = kj_point + sg_point + sf_point + sdm_point # negative point

    # positive point 
    fiber = final_recipes.at[i, 'fiber_100g']
    protien = final_recipes.at[i, 'protein_g']
    # fiber
    if fiber > 4.7:
        fb_point = 5
    else:
        fb_level = fiber % 0.9
        if fb_level <= 0.9:
            fb_point = fiber // 0.9
        else:
            fb_point = (fiber // 0.9) + 1
    # protien
    if protien > 8:
        pr_point = 5
    else:
        pr_level = protien % 1.6
        if pr_level <= 1.6:
            pr_point = protien // 1.6
        else:
            pr_point = (protien // 1.6) + 1
    if N_point > 11:
        P_point = fb_point
    else:
        P_point = fb_point + pr_point

    # Nutri score
    Score = N_point - P_point
#     print(Score)
    final_recipes.at[i, 'Nutri_score'] = NutriScore_string(Score) # fct above

In [113]:
final_recipes.Nutri_score.value_counts().sort_index()

A    11425
B     7215
C     6348
D     4552
E      953
Name: Nutri_score, dtype: int64

In [114]:
# read recipes with ratings
rating_profile = pd.read_csv('Rating-Profile_Merged.csv', sep=';')
rating_profile = rating_profile.rename(columns = {'name':'Name'})
rating_profile = rating_profile[['userid','recipeid','rating','Name']]
rating_profile.columns

Index(['userid', 'recipeid', 'rating', 'Name'], dtype='object')

In [115]:
final_recipes = final_recipes.merge(rating_profile, on = 'Name')
final_recipes.category.value_counts()

Meat and Poultry         39332
Fruits and Vegetables    17328
Pasta and Noodles         5924
Barbecue                  5665
Seafood                   5382
Salad                       68
Name: category, dtype: int64

In [116]:
final_recipes.columns

Index(['id', 'URL', 'Name', 'fiber_g', 'sodium_g', 'carbohydrates_g', 'fat_g',
       'protein_g', 'sugar_g', 'saturate_g', 'size_g', 'Servings',
       'calories_kCal', 'Average Rating', 'Average Sentiment', 'NumberRatings',
       'Number of Bookmarks', 'Year of Publishing', 'category', 'image_link',
       'fiber_100g', 'fat_100g', 'sugar_100g', 'saturated_100g',
       'sodium_100mg', 'carbohydrates_100g', 'protein_100g', 'kj_100g',
       'salt_g', 'salt_100g', 'Fsa_new', 'fat_count', 'satfat_count',
       'sugar_count', 'salt_count', 'Nutri_score', 'userid', 'recipeid',
       'rating'],
      dtype='object')

In [117]:
final_recipes = final_recipes[['recipeid','userid','rating','URL','Name','fiber_g','sodium_g','carbohydrates_g','fat_g','protein_g','sugar_g','saturate_g', 'size_g','Servings',
                  'calories_kCal','category','image_link','fat_100g','fiber_100g','sugar_100g','saturated_100g','protein_100g','sodium_100mg',  
                  'carbohydrates_100g','kj_100g','Nutri_score','Fsa_new','salt_100g','salt_g','fat_count','satfat_count','sugar_count','salt_count','NumberRatings']]

### Split based on Fsa score, with a thershold 8

In [118]:
final_healthy = final_recipes[final_recipes.Fsa_new <= 8]
final_unhealthy = final_recipes[final_recipes.Fsa_new > 8]

### Split based on category

In [119]:
def split_data(category):
    h_cat_data = final_healthy[final_healthy.category == category][['recipeid','userid','rating']]
    unh_cat_data = final_unhealthy[final_unhealthy.category == category][['recipeid','userid','rating']]
    
    # save data h/unh rating matrix
    h_cat_data.to_csv('./recipe_recommender/rating_matrix/h_'+category.replace(' ','_')+'.csv', index=False, header=False)
    unh_cat_data.to_csv('./recipe_recommender/rating_matrix//unh_'+category.replace(' ','_')+'.csv', index=False, header=False)
    
    return "ok :)"

In [120]:
for cat in final_healthy.category.value_counts().index:
    split_data(cat)

In [121]:
final_healthy.drop_duplicates(subset=['recipeid'], inplace=True)
final_unhealthy.drop_duplicates(subset=['recipeid'], inplace=True)

In [122]:
final_healthy.columns

Index(['recipeid', 'userid', 'rating', 'URL', 'Name', 'fiber_g', 'sodium_g',
       'carbohydrates_g', 'fat_g', 'protein_g', 'sugar_g', 'saturate_g',
       'size_g', 'Servings', 'calories_kCal', 'category', 'image_link',
       'fat_100g', 'fiber_100g', 'sugar_100g', 'saturated_100g',
       'protein_100g', 'sodium_100mg', 'carbohydrates_100g', 'kj_100g',
       'Nutri_score', 'Fsa_new', 'salt_100g', 'salt_g', 'fat_count',
       'satfat_count', 'sugar_count', 'salt_count', 'NumberRatings'],
      dtype='object')

In [123]:
final_healthy = final_healthy.rename(columns={'recipeid':'id'})
final_unhealthy = final_unhealthy.rename(columns={'recipeid':'id'})

In [124]:
final_healthy.drop(columns=['userid','rating'], inplace=True)
final_unhealthy.drop(columns=['userid','rating'], inplace=True)

In [125]:
final_healthy.to_csv('./Final_recipes/F_V_healthy.csv', index=False, sep=';')
final_unhealthy.to_csv('./Final_recipes/F_V_unhealthy.csv', index=False, sep=';')

# Rating Matrix

In [126]:
h_Pasta_and_Noodles = pd.read_csv('recipe_recommender/rating_matrix/h_Pasta_and_Noodles.csv', names=['rid','uid','rt'])
h_Pasta_and_Noodles.head()

,rid,uid,rt
0,1058,50730,5
1,1058,30077,4
2,1058,78714,4
3,1058,66074,5
4,1058,22345,5


# Prepare to push to database

In [320]:
import pyodbc
import psycopg2

In [321]:
conn = psycopg2.connect("host=localhost dbname=labelingdb user=ayoub password=ayoub1234")
cur = conn.cursor()

In [262]:
import csv 

with open('./Final_recipes/F_V_healthy.csv') as f:
#     reader = csv.reader(f)
#     next(reader)
    next(f)
#     for row in reader:
#         cur.execute(
#         'INSERT into "Labels_Nudges_recipes" Values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)',
#             row
        
#         )
    cur.copy_from(f, 'HealthyRecipe', sep=';',columns=(
                     'id','URL','Name','fiber_g','sodium_g','carbohydrates_g','fat_g','protein_g','sugar_g','saturate_g', 'size_g','Servings',
                  'calories_kCal','category','image_link','fat_100g','fiber_100g','sugar_100g','saturated_100g','protien_100g','sodium_100mg',  
                  'carbohydrates_100g','kj_100g','Nutri_score','Fsa_new','salt_100g','salt_g','fat_count','satfat_count','sugar_count','salt_count'  
                  ))
conn.commit()

In [322]:
import csv 

with open('./Final_recipes/F_V_unhealthy.csv') as f:
#     reader = csv.reader(f)
#     next(reader)
    next(f)
#     for row in reader:
#         cur.execute(
#         'INSERT into "Labels_Nudges_recipes" Values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)',
#             row
        
#         )
    cur.copy_from(f, 'UnhealthyRecipe', sep=';',columns=(
                     'id','URL','Name','fiber_g','sodium_g','carbohydrates_g','fat_g','protein_g','sugar_g','saturate_g', 'size_g','Servings',
                  'calories_kCal','category','image_link','fat_100g','fiber_100g','sugar_100g','saturated_100g','protien_100g','sodium_100mg',  
                  'carbohydrates_100g','kj_100g','Nutri_score','Fsa_new','salt_100g','salt_g','fat_count','satfat_count','sugar_count','salt_count'  
                  ))
conn.commit()

In [323]:
conn.close()